In [116]:
import numpy as np
import scipy
import random

For Questions 15-20, you will play with PLA and pocket algorithm. First, we use an artificial data set to study PLA. The data set is in

https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_15_train.dat

Each line of the data set contains one (xn,yn) with xn∈ℝ4. The first 4 numbers of the line contains the components of xn orderly, the last number is yn.

Please initialize your algorithm with w=0 and take sign(0) as −1.

In [32]:
data = np.genfromtxt('hw1_15_train.dat')
X = data[:, :4]
y = data[:, 4]

In [306]:
class PLA:
    
    def __init__(self):
        pass
        
    def fit(self, X, y, seed=None, learning_rate=1):
        assert len(X.shape) == 2 and len(y.shape) == 1
        assert X.shape[0] == y.shape[0]
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        self.w = np.zeros(X.shape[1])
        
        indices = []
        if seed is None:
            indices = list(range(X.shape[0]))
        else:
            random.seed(seed)
            indices = random.sample(range(X.shape[0]), X.shape[0])
            
        step = 0
        update_count = 0
        last_step = None # the last step it made a mistake
        while True:
            idx = indices[step]
            x = X[idx]
            t = y[idx]
            prediction = np.sign(x.dot(self.w))
            if prediction == 0:
                prediction = -1 # as required in the problem
                
            if prediction != t:
                self.w += (learning_rate * t) * x
                last_step = step
                update_count += 1
            elif last_step == step: # halt when a full cycle meets no mistake
                # for problem 15, 16, 17
                return update_count
            step = (step + 1) % X.shape[0]
    
    def fit_pocket(self, X, y, seed=None, update_bound=50, pocket=True):
        assert len(X.shape) == 2 and len(y.shape) == 1
        assert X.shape[0] == y.shape[0]
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        self.w = np.zeros(X.shape[1])
        
        if not seed is None:
            random.seed(seed)

        update_count = 0
        w_pocket = np.array(self.w)
        while update_count < update_bound:
            idx = random.randint(0, X.shape[0] - 1)
            x = X[idx]
            t = y[idx]
            prediction = np.sign(x.dot(self.w))       
            if prediction != t:
                self.w += t * x
                update_count += 1
                predictions_before = [np.sign(x.dot(w_pocket)) for x in X]
                predictions_after = [np.sign(x.dot(self.w)) for x in X]        
                if sum(predictions_before == y) < sum(predictions_after == y):
                    w_pocket = np.array(self.w)
        if pocket:
            self.w = w_pocket
    
    def predict(self, X):
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        return [np.sign(x.dot(self.w)) for x in X]
    
    def accuracy(self, X, y):
        # for problem 18, 19, 20
        predictions = self.predict(X)
        return sum(predictions == y) / len(y)
pla = PLA()

Implement a version of PLA by visiting examples in the naive cycle using the order of examples in the data set. Run the algorithm on the data set. What is the number of updates before the algorithm halts?

In [232]:
# problem 15
update_count = pla.fit(X, y)
print('PLA halts at update {}'.format(update_count))

PLA halts at update 45


Implement a version of PLA by visiting examples in fixed, pre-determined random cycles throughout the algorithm. Run the algorithm on the data set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average number of updates before the algorithm halts?

In [177]:
# problem 16
avg_update_count = 0
for seed in range(2000):
    avg_update_count += pla.fit(X, y, seed=seed)
avg_update_count /= 2000
print('PLA halts at update {} on average'.format(avg_update_count))

PLA halts at update 39.624 on average



Implement a version of PLA by visiting examples in fixed, pre-determined random cycles throughout the algorithm, while changing the update rule to be

wt+1←wt+ηyn(t)xn(t)
with η=0.5. Note that your PLA in the previous Question corresponds to η=1. Please repeat your experiment for 2000 times, each with a different random seed. What is the average number of updates before the algorithm halts?

In [131]:
# problem 17
avg_update_count = 0
for seed in range(2000):
    avg_update_count += pla.fit(X, y, seed=seed, learning_rate=0.5)
avg_update_count /= 2000
print('PLA halts at update {} on average with learning rate being 0.5'.format(avg_update_count))

PLA halts at update 39.624 on average with learning rate being 0.5


Next, we play with the pocket algorithm. Modify your PLA in Question 16 to visit examples purely randomly, and then add the "pocket" steps to the algorithm. We will use

https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_train.dat

as the training data set D, and

https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_test.dat

as the test set for "verifying'' the g returned by your algorithm (see lecture 4 about verifying). 
The sets are of the same format as the previous one. 

In [178]:
data = np.genfromtxt('hw1_18_train.dat')
X_train = data[:, :4]
y_train = data[:, 4]

In [179]:
data = np.genfromtxt('hw1_18_test.dat')
X_test = data[:, :4]
y_test = data[:, 4]

Run the pocket algorithm with a total of 50 updates on D , and verify the performance of wPOCKET using the test set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

In [307]:
# problem 18
avg_error_rate = 0
for seed in range(2000):
    pla.fit_pocket(X_train, y_train, seed=seed, update_bound=50)
    avg_error_rate += 1 - pla.accuracy(X_test, y_test)
avg_error_rate /= 2000
print('PLA_pocket has an error rate of {} on average'.format(avg_error_rate))

PLA_pocket has an error rate of 0.1314200000000001 on average


Modify your algorithm in Question 18 to return w50 (the PLA vector after 50 updates) instead of w^ (the pocket vector) after 50 updates.

Run the modified algorithm on D, and verify the performance using the test set.

Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

In [310]:
# problem 19
avg_error_rate = 0
for seed in range(2000):
    pla.fit_pocket(X_train, y_train, seed=seed, update_bound=50, pocket=False)
    avg_error_rate += 1 - pla.accuracy(X_test, y_test)
avg_error_rate /= 2000
print('PLA has an error rate of {} on average'.format(avg_error_rate))

PLA has an error rate of 0.37333999999999995 on average


Modify your algorithm in Question 18 to run for 100 updates instead of 50, and verify the performance of wPOCKET using the test set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

In [312]:
# problem 20
avg_error_rate = 0
for seed in range(2000):
    pla.fit_pocket(X_train, y_train, seed=seed, update_bound=100)
    avg_error_rate += 1 - pla.accuracy(X_test, y_test)
avg_error_rate /= 2000
print('PLA_pocket has an error rate of {} on average with update bound being 100'.format(avg_error_rate))

PLA_pocket has an error rate of 0.11512000000000003 on average with update bound being 100
